## Initialization

In [1]:
# TODO: GET THIS STUFF OUT OF HERE
USER="gino@bustelos.com"
PW="GZNaqUP{fD6AhJHXcCJgt%bbK"
COMPETITION="dogs-vs-cats-redux-kernels-edition"
SHORT_NAME="dogsNcats"

In [2]:
raw_data_path = "data/{}/raw".format(SHORT_NAME)
raw_data_path

'data/dogsNcats/raw'

In [3]:
classes = ['dogs','cats']

## Download Data from Kaggle

http://wiki.fast.ai/index.php/Kaggle_CLI

#### Install Kaggle CLI

In [ ]:
!pip install kaggle-cli

#### Download the Dogs N Cats dataset

In [ ]:
!kg config -g -u '$USER' -c '$COMPETITION'
!kg config -g

In [ ]:
!mkdir -p $raw_data_path
!cd $raw_data_path; kg download -p '$PW'

#### Unzip

In [9]:
!cd $raw_data_path; unzip -q test.zip

In [10]:
!cd $raw_data_path; unzip -q train.zip

In [3]:
!ls -al $raw_data_path

total 835580
drwxrwxr-x 4 ubuntu ubuntu      4096 Apr 13 21:23 .
drwxrwxr-x 4 ubuntu ubuntu      4096 Apr 14 17:18 ..
-rw-rw-r-- 1 ubuntu ubuntu    113903 Apr 13 20:17 sample_submission.csv
drwxr-xr-x 2 ubuntu ubuntu    299008 Sep 20  2013 test
-rw-rw-r-- 1 ubuntu ubuntu 284478493 Apr 13 20:14 test.zip
drwxr-xr-x 2 ubuntu ubuntu    802816 Sep 20  2013 train
-rw-rw-r-- 1 ubuntu ubuntu 569918665 Apr 13 20:17 train.zip


## Separate data into cats and dogs
The following cells are modifying the layout of the files on disk to fit what Keras wants... each class to be in its own directory

In [4]:
import os
import pandas as pd
import numpy as np

In [5]:
clean_data_path = "data/{}/clean".format(SHORT_NAME)
clean_data_path

'data/dogsNcats/clean'

### Train Data

In [6]:
all_train = os.listdir(raw_data_path+'/train')

In [7]:
print("There are {} training files".format(len(all_train)))

There are 25000 training files


In [8]:
all_train_df = pd.DataFrame(data=all_train, columns=['name'])

In [9]:
all_train_df.head()

,name
0,dog.88.jpg
1,cat.3285.jpg
2,cat.12344.jpg
3,dog.2373.jpg
4,dog.8854.jpg


In [13]:
all_train_cats = all_train_df[all_train_df['name'].str.contains('cat')]
all_train_dogs = all_train_df[all_train_df['name'].str.contains('dog')]

In [14]:
print("There are {} cats training files".format(len(all_train_cats)))
print("There are {} dogs training files".format(len(all_train_dogs)))

There are 12500 cats training files
There are 12500 dogs training files


### Need to split the train files into Validation and a Sample set
Note that sample is a copy of part of the training, while the train and valid sets are exclusive

In [15]:
# split into 1000 validation and the rest train
# using sample(frac=1) to shuffle the data in the dataframe
valid_cats, train_cats = np.split(all_train_cats.sample(frac=1), [1000])
valid_dogs, train_dogs = np.split(all_train_dogs.sample(frac=1), [1000])

In [16]:
# sample directory will have a similar train & valid directory but 
# using 100 records.
sample_valid_cats, sample_train_cats = np.split(all_train_cats.sample(n=100), [20])
sample_valid_dogs, sample_train_dogs = np.split(all_train_dogs.sample(n=100), [20])

#### Directory Layout
```
train/
  cats/
  dogs/
  
valid/
  cats/
  dogs/
  
sample/
  train/
    cats/
    dogs/
  valid/
    cats/
    dogs/
```

In [17]:
train_data_path = clean_data_path + '/train'
valid_data_path = clean_data_path + '/valid'

sample_train_data_path = clean_data_path + '/sample/train'
sample_valid_data_path = clean_data_path + '/sample/valid'

def mkdirs(root, dirs ):
    for dir in dirs:
        !mkdir -p $root/$dir


In [18]:
# Create the dogs and cats directories
for root in [train_data_path,valid_data_path,sample_train_data_path,sample_valid_data_path]:
    mkdirs(root, classes)

#### copy files into correct locations

In [19]:
def copy_data(src_files, toDir):
    for src_file in src_files:
        !cp $src_file $toDir/.

In [20]:
import functools

train_path_builder = functools.partial(os.path.join, raw_data_path, 'train' )

#### Training data

In [32]:
copy_data(list(train_cats['name'].apply(train_path_builder)), train_data_path+'/cats')
copy_data(list(train_dogs['name'].apply(train_path_builder)), train_data_path+'/dogs')    

In [42]:
!ls -l $train_data_path/cats | grep cat | wc -l

11500


In [43]:
!ls -l $train_data_path/dogs | grep dog | wc -l

11500


#### Validation data

In [48]:
copy_data(list(valid_cats['name'].apply(train_path_builder)), valid_data_path+'/cats')
copy_data(list(valid_dogs['name'].apply(train_path_builder)), valid_data_path+'/dogs')    

In [54]:
!ls -l $valid_data_path/cats | grep cat | wc -l

1000


In [55]:
!ls -l $valid_data_path/dogs | grep dog | wc -l

1000


#### Sample data

In [21]:
copy_data(list(sample_train_cats['name'].apply(train_path_builder)), sample_train_data_path+'/cats')
copy_data(list(sample_train_dogs['name'].apply(train_path_builder)), sample_train_data_path+'/dogs')   
copy_data(list(sample_valid_cats['name'].apply(train_path_builder)), sample_valid_data_path+'/cats')
copy_data(list(sample_valid_dogs['name'].apply(train_path_builder)), sample_valid_data_path+'/dogs')  

In [22]:
!ls -l $sample_train_data_path/cats | grep cat | wc -l

80


In [23]:
!ls -l $sample_train_data_path/dogs | grep dog | wc -l

80


In [24]:
!ls -l $sample_valid_data_path/cats | grep cat | wc -l

20


In [25]:
!ls -l $sample_valid_data_path/dogs | grep dog | wc -l

20


### Test Data
Test data only needs to get copied... 

In [29]:
!mkdir -p $clean_data_path/test
!cp $raw_data_path/test/*.jpg $clean_data_path/test/.

In [30]:
!ls -l $clean_data_path/test | wc -l

12501


### Sample submission

In [32]:
!cp $raw_data_path/sample_submission.csv $clean_data_path/sample_submission.csv 

### Cleanup
Remove the raw data directory to reduce storage on EBS

In [33]:
!ls $raw_data_path

sample_submission.csv  test  test.zip  train  train.zip


In [34]:
!rm -rf $raw_data_path

In [35]:
!ls $raw_data_path

ls: cannot access 'data/dogsNcats/raw': No such file or directory
